In [1]:
import os 
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader,TextLoader
from langchain_community.vectorstores import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.chat_models import ChatTongyi
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnableLambda
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from operator import itemgetter

# 1. 初始化向量存储和检索器
load_dotenv()

file_path_1="/home/wenhao/projects/Hilbert scheme/PDF/Reduced Hilbert schemes for irreducible curve singularities(G. Pfister, J.H.M. Steenbrink).pdf" 
file_path_3="/home/wenhao/projects/Hilbert scheme/PDF/thesis-some chapter.txt" 

# loader_1 = PyPDFLoader(file_path = file_path_1)
loader_3 = TextLoader(file_path = file_path_3)
docs = loader_3.load()
# loader_1.load() + 


from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ". ", "! ", "? ", "; ", "。", "；", "！" "？", " ", ""]
)

splitte_docs = text_splitter.split_documents(docs)
from langchain_community.embeddings import DashScopeEmbeddings

# 初始化 embedding 模型，指定 model="text-embedding-v4"
embeddings = DashScopeEmbeddings(
    model="text-embedding-v4",
    dashscope_api_key= os.getenv('DASHSCOPE_API_KEY')
)

from langchain_community.vectorstores import Chroma

# 向量数据库储存地址
persist_dir = "/home/wenhao/projects/Hilbert scheme/chroma_db"

# 向量数据库， 如果已存在，不需要重复创建
if os.path.exists(persist_dir) and os.listdir(persist_dir):
    db = Chroma(persist_directory=persist_dir, embedding_function=embeddings)
else:
    db = Chroma.from_documents(
        documents=splitte_docs,
        embedding=embeddings,
        persist_directory=persist_dir
    )

retriever = db.as_retriever()


# 2. 初始化LLM
llm = ChatTongyi(model = "qwen-turbo",temperature = 0, api_key=os.getenv("DASHSCOPE_API_KEY"))

# 3. 定义不同轮次的特定prompt
def get_prompt_for_round(round_num):
    """根据对话轮次返回特定prompt"""
    if round_num == 1:
        return "专注于介绍相关数学的基础知识。请用简单易懂的语言回答问题。"
    elif round_num == 2:
        return "专注于深入解释相关数学的概念。请提供更详细的技术细节。"
    elif round_num >= 3:
        return "，专注于解决复杂问题和提供专业见解。请提供深度分析。"
    return "你是一个数学教授，根据上下文回答问题。"


# 4. 构建链式结构
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "你是一个严谨的数学专家。\n"
    #  "用户额外提供了以下参考文档，如何文档非空白内容，请结合该文档和检索结果回答：\n{file}\n\n"
     "检索到的相关上下文如下：\n{context}\n\n"
     "如果以上内容不包含答案，请回答：'根据已有资料无法回答。'" "{system_prompt}"
    ),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}")

])


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 创建链
chain = (
{     
    "context": itemgetter("question") | retriever | format_docs,
    "system_prompt": RunnableLambda(itemgetter("round_num")) | RunnableLambda(get_prompt_for_round),
    "history": itemgetter("history"),
    "question": itemgetter("question"),
    "file": itemgetter("file")
}
    | prompt_template
    | llm
)

# 5. 创建对话历史管理器
store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_chain = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history"
)

# 6. 运行多轮对话示例
session_id = "user_session_1"


/tmp/ipykernel_18824/1339943457.py:50: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  db = Chroma(persist_directory=persist_dir, embedding_function=embeddings)


In [2]:
file_path_2=  "/home/wenhao/projects/Hilbert scheme/PDF/Continuity-3.txt" 

loader_2 = TextLoader(file_path = file_path_2)
full_docs = loader_2.load()                         # 返回 Document 对象列表

# 合并所有页面文本
pdf_text = "\n".join([page.page_content for page in full_docs])


llm_rewriting_query = ChatTongyi(
    model="qwen3-1.7b",
    temperature=0,
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    model_kwargs={"enable_thinking": False}
)


prompt_rewrite = ChatPromptTemplate.from_messages([
    ("system", 
     "你是一个查询改写助手。请根据对话历史，将用户最新的问题改写成一个独立、完整、清晰的问题。\n"
     "不要回答问题，不要添加解释，只输出改写后的问题。"
    ),
    ("user", "{question}")
])

def rewriting_query(question: str) -> str:
    chain = prompt_rewrite | llm_rewriting_query
    response = chain.invoke({"question": question})
    return response.content.strip()  


In [3]:
query = "Hilbert scheme 是什么？"
rewritten_query = rewriting_query(query)

loader_2 = TextLoader(file_path = file_path_2)
full_docs = loader_2.load()                         # 返回 Document 对象列表

# 合并所有页面文本
pdf_text = "\n".join([page.page_content for page in full_docs])

# 第一轮对话
response = conversational_chain.invoke(
    {      "file": pdf_text, 
        "question": rewritten_query,
        "round_num": 1,
           
    },
    config={"configurable": {"session_id": session_id}}
)
print(f"轮次1: {response}\n")

轮次1: content='Hilbert scheme 是数学中一个重要的概念，主要用于研究几何对象（如曲线、曲面等）的子集。它是一个“参数空间”，也就是说，它用来“收集”和“分类”满足某种条件的子集。\n\n具体来说，Hilbert scheme 可以理解为：  \n**它是一个空间，其中的每一个点对应一个特定类型的子集（比如一个曲线上的点、一个代数曲线的子曲线等），并且这些子集满足某些固定条件（比如长度、维度、多项式等）。**\n\n举个简单的例子，想象你有一个曲线 $ C $，Hilbert scheme 可以用来研究这个曲线上所有可能的“有限子集”（比如一些点组成的集合），每个这样的子集都有一个“长度”（比如有多少个点）。Hilbert scheme 就是把这些长度相同的子集都放在一起，形成一个空间。\n\n在更具体的语境中，比如“punctual Hilbert scheme”，它专门研究的是**在一点附近**（比如一个奇点附近）的有限子集，这些子集的长度是固定的。\n\n总结一下：  \n**Hilbert scheme 是一个用来参数化满足某些条件的子集（如点、曲线等）的空间，它在代数几何中有广泛应用。**' additional_kwargs={} response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'stop', 'request_id': '15a44a96-04cb-4238-b830-207ed583f415', 'token_usage': {'input_tokens': 543, 'output_tokens': 273, 'prompt_tokens_details': {'cached_tokens': 0}, 'total_tokens': 816}} id='lc_run--019bb121-1bfe-7350-b062-bb34fc302441-0' tool_calls=[] invalid_tool_calls=[]



In [4]:
store["user_session_1"].messages

[HumanMessage(content='Hilbert scheme 是什么？', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hilbert scheme 是数学中一个重要的概念，主要用于研究几何对象（如曲线、曲面等）的子集。它是一个“参数空间”，也就是说，它用来“收集”和“分类”满足某种条件的子集。\n\n具体来说，Hilbert scheme 可以理解为：  \n**它是一个空间，其中的每一个点对应一个特定类型的子集（比如一个曲线上的点、一个代数曲线的子曲线等），并且这些子集满足某些固定条件（比如长度、维度、多项式等）。**\n\n举个简单的例子，想象你有一个曲线 $ C $，Hilbert scheme 可以用来研究这个曲线上所有可能的“有限子集”（比如一些点组成的集合），每个这样的子集都有一个“长度”（比如有多少个点）。Hilbert scheme 就是把这些长度相同的子集都放在一起，形成一个空间。\n\n在更具体的语境中，比如“punctual Hilbert scheme”，它专门研究的是**在一点附近**（比如一个奇点附近）的有限子集，这些子集的长度是固定的。\n\n总结一下：  \n**Hilbert scheme 是一个用来参数化满足某些条件的子集（如点、曲线等）的空间，它在代数几何中有广泛应用。**', additional_kwargs={}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'stop', 'request_id': '15a44a96-04cb-4238-b830-207ed583f415', 'token_usage': {'input_tokens': 543, 'output_tokens': 273, 'prompt_tokens_details': {'cached_tokens': 0}, 'total_tokens': 816}}, id='lc_run--019bb121-1bfe-7350-b062-bb34fc302441-0', tool_calls=[], invalid_tool_calls=[])]

In [5]:
# file_path_2=  "/home/wenhao/projects/Hilbert scheme/PDF/Continuity-3.txt" 


query = "	现在谈谈量子力学。"
rewritten_query = rewriting_query(query)


# 第一轮对话
response = conversational_chain.invoke(
    {      "file": "", 
        "question": rewritten_query,
        "round_num": 2,
           
    },
    config={"configurable": {"session_id": session_id}}
)
print(f"轮次1: {response}\n")


轮次1: content='量子力学是描述微观世界中物质和能量行为的物理理论，它与经典物理学有根本的不同。其核心思想是：在微观尺度上，粒子（如电子、光子等）的行为不能用确定性的轨迹来描述，而是需要用概率性的方式进行描述。\n\n### 1. **基本概念**\n\n#### (a) 波粒二象性\n量子力学的一个核心原理是“波粒二象性”，即微观粒子（如光子、电子）既表现出粒子性质，也表现出波动性质。例如：\n- 光在干涉实验中表现出波动性；\n- 在光电效应中又表现出粒子性（光子）。\n\n#### (b) 量子态与波函数\n在量子力学中，一个系统的状态由**波函数** $\\psi(x, t)$ 描述，它是复数函数，满足薛定谔方程：\n\n$$\ni\\hbar \\frac{\\partial}{\\partial t} \\psi(x, t) = \\hat{H} \\psi(x, t),\n$$\n\n其中 $\\hat{H}$ 是系统的**哈密顿算符**（代表总能量），$\\hbar$ 是约化普朗克常数。\n\n波函数的绝对值平方 $|\\psi(x, t)|^2$ 表示在位置 $x$ 和时间 $t$ 处找到粒子的概率密度。\n\n#### (c) 算符与可观测量\n在量子力学中，物理量（如位置、动量、能量等）由**线性算符**表示。例如：\n- 位置算符：$\\hat{x}$\n- 动量算符：$\\hat{p} = -i\\hbar \\frac{d}{dx}$\n\n这些算符作用于波函数上，可以得到对应的物理量的期望值或可能的测量结果。\n\n#### (d) 不确定性原理\n海森堡不确定性原理指出，某些物理量对（如位置和动量、时间和能量）不能同时被精确测量。例如：\n\n$$\n\\Delta x \\cdot \\Delta p \\geq \\frac{\\hbar}{2},\n$$\n\n其中 $\\Delta x$ 是位置的不确定度，$\\Delta p$ 是动量的不确定度。\n\n---\n\n### 2. **数学结构**\n\n量子力学可以用**希尔伯特空间**（Hilbert space）来形式化。这是一个复数向量空间，配备内积（inner product），允许我们定义正交性和归一化条件。\n\n#### (a) 状态向量\n量子态可

In [6]:
store["user_session_1"].messages

[HumanMessage(content='Hilbert scheme 是什么？', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hilbert scheme 是数学中一个重要的概念，主要用于研究几何对象（如曲线、曲面等）的子集。它是一个“参数空间”，也就是说，它用来“收集”和“分类”满足某种条件的子集。\n\n具体来说，Hilbert scheme 可以理解为：  \n**它是一个空间，其中的每一个点对应一个特定类型的子集（比如一个曲线上的点、一个代数曲线的子曲线等），并且这些子集满足某些固定条件（比如长度、维度、多项式等）。**\n\n举个简单的例子，想象你有一个曲线 $ C $，Hilbert scheme 可以用来研究这个曲线上所有可能的“有限子集”（比如一些点组成的集合），每个这样的子集都有一个“长度”（比如有多少个点）。Hilbert scheme 就是把这些长度相同的子集都放在一起，形成一个空间。\n\n在更具体的语境中，比如“punctual Hilbert scheme”，它专门研究的是**在一点附近**（比如一个奇点附近）的有限子集，这些子集的长度是固定的。\n\n总结一下：  \n**Hilbert scheme 是一个用来参数化满足某些条件的子集（如点、曲线等）的空间，它在代数几何中有广泛应用。**', additional_kwargs={}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'stop', 'request_id': '15a44a96-04cb-4238-b830-207ed583f415', 'token_usage': {'input_tokens': 543, 'output_tokens': 273, 'prompt_tokens_details': {'cached_tokens': 0}, 'total_tokens': 816}}, id='lc_run--019bb121-1bfe-7350-b062-bb34fc302441-0', tool_calls=[], invalid_tool_calls=[]),
 HumanMessage

In [7]:

query = "它有什么应用？"
rewritten_query = rewriting_query(query)

	

# loader_2 = TextLoader(file_path = file_path_2)
# full_docs = loader_2.load()                         # 返回 Document 对象列表

# 合并所有页面文本
pdf_text = "\n".join([page.page_content for page in full_docs])

# 第一轮对话
response = conversational_chain.invoke(
    {      "file": "", 
        "question": rewritten_query,
        "round_num": 3,
           
    },
    config={"configurable": {"session_id": session_id}}
)
print(f"轮次1: {response}\n")


轮次1: content='量子力学在现代科学和技术中有着极其广泛的应用，其理论框架不仅解释了微观世界的物理现象，还为许多前沿技术提供了基础。以下是一些主要的、具有代表性的应用领域：\n\n---\n\n## 1. **原子与分子物理**\n\n- **原子结构**：量子力学成功地解释了氢原子和多电子原子的能级结构（如玻尔模型、薛定谔方程求解）。\n- **光谱学**：通过分析原子和分子的发射与吸收光谱，可以识别物质成分，用于化学分析、天体物理等。\n- **化学键理论**：量子力学是理解化学反应机理和分子结构的基础，如价键理论、分子轨道理论等。\n\n---\n\n## 2. **凝聚态物理**\n\n- **固体物理**：量子力学是理解金属、半导体、绝缘体、超导体等材料性质的基础。\n    - **能带理论**：解释电子在晶体中的行为，为半导体器件（如晶体管、二极管）的设计提供理论依据。\n- **超导与超流**：量子效应在低温下导致宏观尺度上的无电阻电流或无粘滞流动。\n- **拓扑材料**：基于量子力学的拓扑相变理论，发展出新型材料如拓扑绝缘体、拓扑超导体。\n\n---\n\n## 3. **核物理与粒子物理**\n\n- **核反应与衰变**：量子隧穿效应解释了放射性衰变（如α衰变）。\n- **粒子加速器与对撞机**：基于量子场论（QFT）和标准模型，研究基本粒子的相互作用。\n- **中微子物理**：量子力学描述中微子振荡现象，揭示粒子质量非零的特性。\n\n---\n\n## 4. **量子信息科学**\n\n- **量子计算**：利用量子比特（qubit）的叠加和纠缠特性，实现超越经典计算机的计算能力。\n    - 例如，Shor算法可高效分解大整数，威胁传统加密体系。\n- **量子通信**：利用量子纠缠实现安全的信息传输（如量子密钥分发）。\n- **量子密码学**：基于量子不可克隆定理，构建无法被窃听的通信协议。\n\n---\n\n## 5. **光学与光子学**\n\n- **激光技术**：基于受激辐射原理，量子力学是激光工作的理论基础。\n- **量子光学**：研究光子与物质的相互作用，包括单光子源、量子态操控等。\n- **量子成像**：利用量子纠缠光子实现超越经典极限的成像技术（如鬼成像）。\n\n---\n\n## 6. *

In [8]:
rewritten_query

'它有哪些应用？'

In [13]:
store["user_session_1"].messages

[HumanMessage(content='Hilbert scheme 是什么？', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hilbert scheme 是一个数学对象，用来“参数化”某种几何结构。具体来说，它是一个空间（可以理解为一种特殊的几何对象），其中的每个点对应于某个特定几何对象的一个子结构。\n\n例如，在代数几何中，Hilbert scheme 可以用来参数化某个代数曲线或代数簇的所有子集，这些子集满足某些特定的条件，比如它们的“长度”或“维数”。\n\n举个简单的例子：假设你有一个代数曲线 $ C $，Hilbert scheme $ C^{[\\ell]} $ 就是所有长度为 $ \\ell $ 的“0维子概形”（可以理解为一些点的集合，每个点有某种“权重”或“多重性”）的集合。每个这样的子概形都可以看作是曲线 $ C $ 上的一个“有限子集”，但允许有重复或重叠。\n\nHilbert scheme 在现代数学中非常重要，特别是在研究代数几何、模空间和相关不变量时。', additional_kwargs={}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'stop', 'request_id': 'b4eb2374-728b-4fa1-b593-8c4339a188b7', 'token_usage': {'input_tokens': 543, 'output_tokens': 217, 'prompt_tokens_details': {'cached_tokens': 0}, 'total_tokens': 760}}, id='lc_run--019bb11b-10c4-7181-bad6-5d06a7949d2a-0', tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='\t现在谈谈量子力学。', additional_kwargs={}, response_metadata={}),
 AIMessage(content='量子力学是描述微观粒子（如电子、光子等）行为的物理理论。它与

In [ ]:
query = "它的证明有多长？"
rewritten_query = rewriting_query(query)

	

# loader_2 = TextLoader(file_path = file_path_2)
# full_docs = loader_2.load()                         # 返回 Document 对象列表

# 合并所有页面文本
pdf_text = "\n".join([page.page_content for page in full_docs])

# 第一轮对话
response = conversational_chain.invoke(
    {      "file": "", 
        "question": query,
        "round_num": 4,
           
    },
    config={"configurable": {"session_id": session_id}}
)
print(f"轮次1: {response}\n")


轮次1: content="这是一个非常有趣的问题！\n\n安德鲁·怀尔斯（Andrew Wiles）在1994年证明费马大定理的论文，**长度相当长**，大约有**100多页**。不过，这还不是全部。\n\n### 1. **怀尔斯的原始论文**\n- 怀尔斯最初的论文发表于1995年的《数学年刊》（*Annals of Mathematics*），题为：\n  > **Modular elliptic curves and Fermat's Last Theorem**  \n  由安德鲁·怀尔斯撰写。\n- 这篇论文本身约有**100多页**，包含了大量复杂的数学内容，涉及**模形式、椭圆曲线、伽罗瓦表示、Iwasawa理论等**。\n\n### 2. **后续补充和修正**\n- 在最初提交时，怀尔斯发现了一个漏洞（即所谓的“错误”），他与他的学生**理查德·泰勒**（Richard Taylor）一起进行了长达一年的修正工作。\n- 修正后的版本被重新提交，并最终在1995年正式发表。\n\n### 3. **更广泛的背景和解释**\n- 如果你想要完全理解怀尔斯的证明，还需要阅读一些相关的背景材料，比如：\n  - 谷山-志村猜想（Taniyama–Shimura conjecture）\n  - 模形式与椭圆曲线之间的关系\n  - 伽罗瓦表示和模性提升（modularity lifting）技术\n\n这些内容加起来可能需要**几十本书或数百页的资料**才能完整掌握。\n\n---\n\n### 总结：\n\n- **怀尔斯原始论文的长度：约100页**\n- **要完全理解其证明所需的总知识量：远超100页，可能达到数百页甚至更多**\n\n如果你对其中某些部分感兴趣，我可以为你详细讲解！" additional_kwargs={} response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'stop', 'request_id': '2a3bb6a8-d179-4e4a-845d-c4ead4514cf0', 'token_usage': {'input_tokens': 1217, 'output_tokens': 396, 'prompt_tokens_details': 

In [10]:


query = "请解释哥德尔不完备定理。"
rewritten_query = rewriting_query(query)

	

# loader_2 = TextLoader(file_path = file_path_2)
# full_docs = loader_2.load()                         # 返回 Document 对象列表

# 合并所有页面文本
pdf_text = "\n".join([page.page_content for page in full_docs])

# 第一轮对话
response = conversational_chain.invoke(
    {      "file": "", 
        "question": query,
        "round_num": 2,
           
    },
    config={"configurable": {"session_id": session_id}}
)
print(f"轮次1: {response}\n")


轮次1: content="哥德尔不完备定理（**Gödel's Incompleteness Theorems**）是20世纪初由奥地利数学家**库尔特·哥德尔（Kurt Gödel）**提出的两个深刻而重要的数学逻辑结果。它们对数学基础、形式系统和人类知识的边界产生了深远影响。\n\n---\n\n## 🧠 一、哥德尔不完备定理的核心思想\n\n哥德尔在1931年发表的论文《论《数学原理》及相关系统中的形式不可判定命题》（*Über formal unentscheidbare Sätze der Principia Mathematica und verwandter Systeme*）中，提出了以下两个定理：\n\n---\n\n## ✅ **第一不完备定理（First Incompleteness Theorem）**\n\n> **任何包含初等算术的形式系统，如果它是**一致的（即没有矛盾），那么它就是**不完全的**（即存在一些命题在这个系统中无法被证明或证伪）。**\n\n### 更通俗地解释：\n- 如果一个数学系统足够强大（比如可以表达基本的算术），并且没有内在矛盾（一致），那么这个系统就**不能证明所有真命题**。\n- 换句话说，**总有一些命题既不能从系统中推出，也不能被系统否定**。\n\n---\n\n## ✅ **第二不完备定理（Second Incompleteness Theorem）**\n\n> **任何包含初等算术的一致形式系统，**不能在系统内部证明它自身的无矛盾性（一致性）**。**\n\n### 更通俗地解释：\n- 你不能在一个数学系统内用它自己的规则来证明“这个系统不会产生矛盾”。\n- 这意味着，**我们无法通过系统自身来确认它的可靠性**。\n\n---\n\n## 🧩 二、为什么这些定理重要？\n\n哥德尔的不完备定理对数学、哲学、计算机科学等多个领域都产生了巨大影响：\n\n### 1. **对数学基础的冲击**\n- 哥德尔的工作挑战了当时流行的**形式主义**（如希尔伯特计划）——即希望通过形式化公理系统来建立一套完备、一致、可判定的数学体系。\n- 它表明，**数学不可能完全形式化**，有些真理超出了形式系统的范围。\n\n### 2. **对逻辑与计算理论的影响**\n- 为后来的**图灵机**、*

In [11]:

query = "该定理对数学基础有何影响？"
rewritten_query = rewriting_query(query)

	

# loader_2 = TextLoader(file_path = file_path_2)
# full_docs = loader_2.load()                         # 返回 Document 对象列表

# 合并所有页面文本
pdf_text = "\n".join([page.page_content for page in full_docs])

# 第一轮对话
response = conversational_chain.invoke(
    {      "file": "", 
        "question": query,
        "round_num": 2,
           
    },
    config={"configurable": {"session_id": session_id}}
)
print(f"轮次1: {response}\n")


轮次1: content="哥德尔不完备定理（**Gödel's Incompleteness Theorems**）对数学基础产生了深远而根本性的影响，主要体现在以下几个方面：\n\n---\n\n## 一、**挑战了形式主义的数学哲学**\n\n### 1. **希尔伯特计划的失败**\n- 在20世纪初，德国数学家**大卫·希尔伯特**提出了一个雄心勃勃的计划，称为“**希尔伯特计划**”（Hilbert's Program），其目标是：\n  - 将整个数学形式化为一组公理系统；\n  - 证明这些系统的一致性（无矛盾）；\n  - 证明它们的完备性（所有真命题都可以被证明）。\n\n### 2. **哥德尔定理的冲击**\n- 哥德尔的第一不完备定理表明：**任何足够强大的形式系统，如果是一致的，那么它就是不完全的**。\n- 第二不完备定理进一步指出：**这样的系统无法在自身内部证明自己的无矛盾性**。\n- 这意味着**希尔伯特计划的目标不可能实现**。数学不能通过形式化手段完全自我证明和完备。\n\n---\n\n## 二、**揭示了数学真理的不可穷尽性**\n\n### 1. **存在不可判定的命题**\n- 哥德尔构造了一个在系统中既不能被证明也不能被否定的命题（即“自指语句”）。\n- 这表明，**数学中存在一些命题是“真但不可证”的**，也就是说，**数学的真理超出了形式系统的范围**。\n\n### 2. **数学不是封闭的系统**\n- 数学不再是“可以由有限公理推出一切”的封闭结构。\n- 它是一个开放的、不断扩展的领域，有些问题可能永远无法用现有的形式系统解决。\n\n---\n\n## 三、**改变了对逻辑与计算的理解**\n\n### 1. **启发图灵机与计算机科学**\n- 哥德尔的工作为后来的**艾伦·图灵**（Alan Turing）研究**停机问题**（Halting Problem）奠定了基础。\n- 图灵证明了某些问题在计算上是不可解的，这与哥德尔的不完备定理有深刻的联系。\n\n### 2. **逻辑与计算理论的结合**\n- 哥德尔定理促使人们更深入地思考**逻辑系统、可计算性、递归函数**等概念之间的关系。\n- 今天，这些思想在**人工智能、自动定理证明、编程语言理论**等领域仍然具有重要意义。\n\n---